# Advanced Agent Tutorial: Human-in-the-Loop, Streaming, and Tracing

This tutorial demonstrates advanced AdalFlow agent features including permission management, real-time streaming, and comprehensive tracing capabilities.

## Prerequisites
1. Install AdalFlow: `pip install adalflow`
2. Set OpenAI API key: `export OPENAI_API_KEY="your-api-key-here"`

**Note**: This notebook requires an OpenAI API key to function properly.

In [ ]:
from IPython.display import clear_output 


!pip install adalflow pydantic openai

# Set OpenAI API key (replace with your actual key)
import os
os.environ["OPENAI_API_KEY"] = "your api key"
# Optional: Set Anthropic API key for Anthropic examples
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"
clear_output()

## Setup and Imports

In [3]:
# Import required libraries
import os
import sys
import asyncio
from adalflow.components.agent.agent import Agent
from adalflow.components.agent.runner import Runner
from adalflow.components.model_client import OpenAIClient
from adalflow.components.model_client.anthropic_client import AnthropicAPIClient
from adalflow.core.func_tool import FunctionTool
from adalflow.core.types import (
    ToolOutput, 
    RawResponsesStreamEvent, 
    RunItemStreamEvent,
    ToolCallRunItem,
    ToolOutputRunItem,
    FinalOutputItem,
    ToolCallActivityRunItem
)
from adalflow.apps.cli_permission_handler import CLIPermissionHandler, AutoApprovalHandler
from adalflow.tracing import (
    set_tracing_disabled, 
    enable_mlflow_local, 
    trace
)
import adalflow as adal
from pathlib import Path
import os

ENV_PATH = Path(os.getcwd()).parent.parent / ".env"
print(ENV_PATH)

# Setup environment
adal.setup_env(ENV_PATH)

/Users/liyin/Documents/test/adal/backend/AdalFlow/.env


In [4]:
adal.__version__

'1.1.0'

## Hello World Agent

Our agent supports sync, async, sync generator and async generator functions whether it uses tooloutput or not.

In [10]:
# Demonstration of all 4 function types supported by AdalFlow agents

def calculator(expression: str) -> str:
    """A synchronous calculator that evaluates mathematical expressions."""
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

# 2. Async Function
async def web_search(query: str="what is the weather in SF today?") -> str:
    """web search on query."""
    # Simulate async operation
    await asyncio.sleep(0.5)
    return "San Francisco will be mostly cloudy today with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight."
    
# streaming tool can include toocallactivity run item and the last one as the final output observable to the llm agent
# 3. Sync Generator Function
def counter(limit: int):
    """A counter that counts up to a limit."""
    final_output = []
    for i in range(1, limit + 1):
        stream_item =  f"Count: {i}/{limit}"
        final_output.append(stream_item)
        stream_event = ToolCallActivityRunItem(data=stream_item)
        yield stream_event

    yield final_output

# 4. Async Generator Function
async def data_streamer(data_type: str):
    """Streams different types of data.

    Args:
        data_types can be "numbers", "letters", "words"
    """
    import asyncio
    data_items = {
        "numbers": [1, 2, 3, 4, 5],
        "letters": ["a", "b", "c", "d", "e"],
        "words": ["hello", "world", "from", "async", "generator"]
    }
    
    items = data_items.get(data_type, ["unknown"])
    final_output = []
    for item in items:
        await asyncio.sleep(0.2)  # Simulate async data fetching
        stream_item =  f"Streaming {data_type}: {item}"
        final_output.append(stream_item)
        stream_event = ToolCallActivityRunItem(data=stream_item)
        yield stream_event

    yield final_output


In [6]:
# setup the agent 

agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), web_search, counter, data_streamer],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

Prompt kwargs: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs after conversion: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n    "kwargs": "The keyword arguments of the function (Optional[Dict[str, object]]) (optional)",\n    "_is_answer_final": "Whether this current output is the final answer (Optional[bool]) (optional)",\n    "_answer": "The final answer if this is the final output. (Optional[typing.Any]) (optional)"\n}'}
Prompt kwargs after conversion: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n   

In [7]:
query = "What is 15 * 7 + 23? What is the sf weather, count to 10, and stream different types of data and tell me what is streamed"
 # query = "what is the sf weahter"

In [8]:
# run call 


# Note: it should work on both sync and async fun normally, but cant work in the notebook, so we test only sync function 


agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), counter],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

def test():
    runner_result = runner.call( prompt_kwargs={"input_str": query})
    print(runner_result)
    print(runner_result.answer)

test()

Prompt kwargs: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs after conversion: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n    "kwargs": "The keyword arguments of the function (Optional[Dict[str, object]]) (optional)",\n    "_is_answer_final": "Whether this current output is the final answer (Optional[bool]) (optional)",\n    "_answer": "The final answer if this is the final output. (Optional[typing.Any]) (optional)"\n}'}
Prompt kwargs after conversion: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n   

In [11]:
# run acall

agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), web_search, counter, data_streamer],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

runner_result = await runner.acall( prompt_kwargs={"input_str": query})

print(runner_result)
print(runner_result.answer)

Prompt kwargs: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs after conversion: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n    "kwargs": "The keyword arguments of the function (Optional[Dict[str, object]]) (optional)",\n    "_is_answer_final": "Whether this current output is the final answer (Optional[bool]) (optional)",\n    "_answer": "The final answer if this is the final output. (Optional[typing.Any]) (optional)"\n}'}
Prompt kwargs after conversion: {'example': '', 'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n   

In [13]:
# run async streaming with streaming events

async def run_astrem():
    streaming_result = runner.astream(
            prompt_kwargs={"input_str": query},
            model_kwargs={"stream": True}
    )

    # Process streaming events
    async for event in streaming_result.stream_events():
        if isinstance(event, RunItemStreamEvent):
            if isinstance(event.item, ToolCallRunItem):
                print(f"\n🔧 Calling tool: {event.item.data.name}")
                
            elif isinstance(event.item, ToolOutputRunItem):
                print(f"✅ Tool output: {event.item.data.output}")
                
            elif isinstance(event.item, ToolCallActivityRunItem):
                # This captures generator yields
                print(f"📝 Activity: {event.item.data}")
                
            elif isinstance(event.item, FinalOutputItem):
                print(f"\n🎯 Final answer: {event.item.data.answer}")

    return streaming_result

asyncio.run(run_astrem())


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


Prompt kwargs: {'context_variables': None, 'task_desc': Parameter(name=react_agent_task_desc, requires_opt=True, param_type=prompt (Instruction to the language model on task, data, and format.), role_desc=Task instruction for the agent to plan steps to solve a question in sequential and multi-steps to get the final answer.             For optimizer: you need to adapt this to the current specific task., data=
<START_OF_TASK_SPEC>
You are an excellent task planner.

Answer the input query using the tools provided below with maximum accuracy.

Each step you will read the previous thought, Action(name, kwargs), and Observation(execution result of the action) and then provide the next Thought and Action
following the output format in <START_OF_OUTPUT_FORMAT> <END_OF_OUTPUT_FORMAT>.

Follow function docstring to best call the tool.
    - For simple queries: You can directly answer by setting `_is_answer_final` to True and generate the answer in the `_answer` field.
    - For complex queries:

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


2025-07-31 19:37:58 - [runner.py:872:impl_astream] - function: Function(id='3dbadcd4-bc11-438a-b5b3-e381da75ec6b', thought='Next, I will check the weather in San Francisco.', name='web_search', args=[], kwargs={'query': 'what is the weather in SF today?'}, _is_answer_final=False, _answer=None)
permission not required
2025-07-31 19:37:58 - [tool_manager.py:431:execute_func_async] - Executing async function: web_search
output arguments () {'query': 'what is the weather in SF today?'}

🔧 Calling tool: web_search
2025-07-31 19:37:59 - [tool_manager.py:446:execute_func_async] - result after await: FunctionOutput(name='web_search', input=Function(id=None, thought=None, name='web_search', args=(), kwargs={'query': 'what is the weather in SF today?'}, _is_answer_final=None, _answer=None), parsed_input=None, output='San Francisco will be mostly cloudy today with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight.', error=None)
Prompt kwargs: {'context_variable

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.
Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


2025-07-31 19:38:00 - [runner.py:872:impl_astream] - function: Function(id='93d1cb4a-b67e-47af-9975-9750b6ccd7af', thought='Now, I will count to 10.', name='counter', args=[], kwargs={'limit': 10}, _is_answer_final=False, _answer=None)
permission not required
2025-07-31 19:38:00 - [tool_manager.py:431:execute_func_async] - Executing async function: counter
output arguments () {'limit': 10}
2025-07-31 19:38:00 - [tool_manager.py:446:execute_func_async] - result after await: FunctionOutput(name='counter', input=Function(id=None, thought=None, name='counter', args=(), kwargs={'limit': 10}, _is_answer_final=None, _answer=None), parsed_input=None, output=<generator object counter at 0x107fb6cf0>, error=None)
Prompt kwargs: {'context_variables': None, 'task_desc': Parameter(name=react_agent_task_desc, requires_opt=True, param_type=prompt (Instruction to the language model on task, data, and format.), role_desc=Task instruction for the agent to plan steps to solve a question in sequential and

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


2025-07-31 19:38:08 - [runner.py:872:impl_astream] - function: Function(id='953041c7-d033-4f0b-8709-62248f633d6c', thought='I have completed all the tasks: calculated the expression, checked the weather, counted to 10, and streamed numbers.', name='', args=[], kwargs={}, _is_answer_final=True, _answer='The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. I counted to 10. The data streamed were numbers: 1, 2, 3, 4, 5.')
2025-07-31 19:38:08 - [runner.py:876:impl_astream] - processed_data: The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. I counted to 10. The data streamed were numbers: 1, 2, 3, 4, 5.

🎯 Final answer: The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) an

RunnerStreamingResult(_event_queue=<Queue at 0x107aa94f0 maxsize=0 _queue=[QueueCompleteSentinel()] tasks=1>, _run_task=<Task finished name='Task-7' coro=<Runner.impl_astream() done, defined at /Users/liyin/Documents/test/adal/backend/AdalFlow/adalflow/adalflow/components/agent/runner.py:770> result=None>, _exception=None, answer='The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. I counted to 10. The data streamed were numbers: 1, 2, 3, 4, 5.', step_history=[StepOutput(step=0, action=Function(id='a4afe886-7317-471a-9e96-662a93d6bdf1', thought='First, I will calculate the mathematical expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='a4afe886-7317-471a-9e96-662a93d6bdf1', thought='First, I will calculate the mathematical expression 15 * 7 + 23.', name='calculator', 

### Streaming Raw Responses

In [7]:
async def streaming_raw_responses_example():
    """Demonstrates handling raw response stream events."""
    print("\n=== Streaming Raw Responses Example ===")
    
    agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator)],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

    runner = Runner(agent=agent)

    streaming_result = runner.astream(
        prompt_kwargs={"input_str": "Calculate 25 * 4 and explain the result"},
        model_kwargs={"stream": True}
    )

    print("Raw streaming output:")
    async for event in streaming_result.stream_events():
        if isinstance(event, RawResponsesStreamEvent):
            # Process raw model output
            if hasattr(event.data, 'choices') and event.data.choices:
                delta = event.data.choices[0].delta
                if hasattr(delta, 'content') and delta.content:
                    print(delta.content, end='', flush=True)

    print("\n")  # Add newline after streaming
    return streaming_result

# Run the async example
await run_async_example_safely(streaming_raw_responses_example, "Streaming Raw Responses")


📋 Running Streaming Raw Responses...

=== Streaming Raw Responses Example ===
Prompt kwargs: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs after conversion: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs: {'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n    "kwargs": "The keyword arguments of the function (Optional[Dict[str, object]]) (optional)",\n    "_is_answer_final": "Whether this current output is the final answer (Optional[bool]) (optional)",\n    "_answer": "The final answer if this is the final output. (Optional[Any]) (optional)"\n}', 'example': ''}
Prompt kwargs after conversion: {'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


2025-07-31 17:00:49 - [runner.py:806:impl_astream] - function: Function(id='b4b309d3-0fb6-4106-aa93-9e964a47656e', thought='First, calculate the expression 25 * 4.', name='calculator', args=[], kwargs={'expression': '25 * 4'}, _is_answer_final=False, _answer=None)
permission not required
2025-07-31 17:00:49 - [tool_manager.py:431:execute_func_async] - Executing async function: calculator
output arguments () {'expression': '25 * 4'}
output in synchronous function call 100
2025-07-31 17:00:49 - [tool_manager.py:446:execute_func_async] - result after await: FunctionOutput(name='calculator', input=Function(id=None, thought=None, name='calculator', args=(), kwargs={'expression': '25 * 4'}, _is_answer_final=None, _answer=None), parsed_input=None, output=100, error=None)
Prompt kwargs: {'step_history': [StepOutput(step=0, action=Function(id='b4b309d3-0fb6-4106-aa93-9e964a47656e', thought='First, calculate the expression 25 * 4.', name='calculator', args=[], kwargs={'expression': '25 * 4'}, _i

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


2025-07-31 17:00:52 - [runner.py:806:impl_astream] - function: Function(id='d50eabf3-054f-452d-a919-669a83529e7e', thought='The calculation of 25 * 4 results in 100. This is because when you multiply 25 by 4, you are essentially adding 25 four times: 25 + 25 + 25 + 25 = 100.', name='', args=[], kwargs={}, _is_answer_final=True, _answer='The result of 25 * 4 is 100. This means that if you have 25 units of something and you multiply it by 4, you end up with 100 units.')
2025-07-31 17:00:52 - [runner.py:810:impl_astream] - processed_data: The result of 25 * 4 is 100. This means that if you have 25 units of something and you multiply it by 4, you end up with 100 units.


✅ Streaming Raw Responses completed successfully


RunnerStreamingResult(_event_queue=<Queue at 0x117580590 maxsize=0 _queue=[QueueCompleteSentinel()] tasks=1>, _run_task=<Task finished name='Task-4' coro=<Runner.impl_astream() done, defined at /Users/liyin/Documents/test/sylphai.com/scripts/.venv/lib/python3.12/site-packages/adalflow/components/agent/runner.py:704> result=None>, _exception=None, answer='The result of 25 * 4 is 100. This means that if you have 25 units of something and you multiply it by 4, you end up with 100 units.', step_history=[StepOutput(step=0, action=Function(id='b4b309d3-0fb6-4106-aa93-9e964a47656e', thought='First, calculate the expression 25 * 4.', name='calculator', args=[], kwargs={'expression': '25 * 4'}, _is_answer_final=False, _answer=None), function=Function(id='b4b309d3-0fb6-4106-aa93-9e964a47656e', thought='First, calculate the expression 25 * 4.', name='calculator', args=[], kwargs={'expression': '25 * 4'}, _is_answer_final=False, _answer=None), observation=100, ctx=None)], _is_complete=True)

### Streaming with Anthropic

In [8]:
async def streaming_anthropic_example():
    """Demonstrates streaming with Anthropic client."""
    print("\n=== Streaming Anthropic Example ===")
    
    # Check if Anthropic API key is available
    if not os.getenv("ANTHROPIC_API_KEY"):
        print("⚠️ ANTHROPIC_API_KEY not set, skipping Anthropic example")
        return None
    
    agent = Agent(
        name="AnthropicAgent",
        tools=[FunctionTool(calculator)],
        model_client=AnthropicAPIClient(),
        model_kwargs={"model": "claude-3-5-haiku-20241022", "stream": True, "temperature": 0.8},
        max_steps=5
    )

    runner = Runner(agent=agent)

    streaming_result = runner.astream(
        prompt_kwargs={"input_str": "Calculate 42 * 3 and explain why this might be significant"},
        model_kwargs={"stream": True}
    )

    async for event in streaming_result.stream_events():
        if isinstance(event, RunItemStreamEvent):
            if isinstance(event.item, FinalOutputItem):
                print(f"🎯 Anthropic Final answer: {event.item.data.answer}")

    return streaming_result

# Run the async example
await run_async_example_safely(streaming_anthropic_example, "Streaming Anthropic")


📋 Running Streaming Anthropic...

=== Streaming Anthropic Example ===
Prompt kwargs: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs after conversion: {'role_desc': 'You are an excellent task planner.'}
Prompt kwargs: {'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": "The name of the function (str) (optional)",\n    "kwargs": "The keyword arguments of the function (Optional[Dict[str, object]]) (optional)",\n    "_is_answer_final": "Whether this current output is the final answer (Optional[bool]) (optional)",\n    "_answer": "The final answer if this is the final output. (Optional[Any]) (optional)"\n}', 'example': ''}
Prompt kwargs after conversion: {'schema': '{\n    "thought": "Your reasoning for this step. Be short for simple queries. For complex queries, provide a clear chain of thought. (Optional[str]) (optional)",\n    "name": 

RunnerStreamingResult(_event_queue=<Queue at 0x1175b22d0 maxsize=0 _queue=[QueueCompleteSentinel()] tasks=1>, _run_task=<Task finished name='Task-7' coro=<Runner.impl_astream() done, defined at /Users/liyin/Documents/test/sylphai.com/scripts/.venv/lib/python3.12/site-packages/adalflow/components/agent/runner.py:704> result=None>, _exception=None, answer="The result of 42 * 3 is 126, which is an interesting number. While 42 itself is famously known from Douglas Adams' 'The Hitchhiker's Guide to Galaxy' as the 'Answer to the Ultimate Question of Life, the Universe, and Everything', the result 126 has some intriguing mathematical properties. It's an even number that is divisible by 2, 3, 6, 7, 9, 14, 18, 21, 42, and 63. In numerology and mathematical curiosities, 126 can be seen as a product that combines the mystique of 42 with the multiplicative power of 3, creating a number that represents complexity and interconnectedness.", step_history=[StepOutput(step=0, action=Function(id='2a12644